# GABO: Gene Analysis Base Optimization
This notebook contains the python code used for getting the results reported in the paper "GABO: Gene Analysis Base Optimization" written by Professors Jonatan Gomez and Elizabeth Leon from Universidad Nacional de Colombia and accepted in the IEEE World Congress on Computational Intelligence - WCCI 2022.

Algorithms are written using the **sgoal** library that can be obtained from the guthub of professor Jonatan Gomez: https://github.com/jgomezpe/. 
To run scripts in this notebook, you must clone the repository running the following script. It will create the *sgoal* folder. If you already cloned it and you do not want to update it, just skip this step:

In [ ]:
!git clone https://github.com/jgomezpe/sgoal.git

Cloning into 'sgoal'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 28 (delta 7), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (28/28), done.


If you have already cloned the repository with the **sgoal** library and you want to retrieve the newest one, run the following script and then run option *Restart runtime* under *Runtime* menu, otherwise skip this step:

In [5]:
%cd /content/sgoal
! git pull 

/content/sgoal
Already up to date.


Now you have to mount the *sgoal* folder and you have to add the python path to the system:

In [6]:
from google.colab import drive
import os, sys

drive.mount('/sgoal', force_remount=True) 
# adding python to the system path
sys.path.insert(0, 'sgoal/python/')

Mounted at /sgoal


Now you can run each one of the algorithms on the testbed functions


##Experiment 1
Comparing GABO, GSC1, and RMHC

In [7]:
from binary import maxones
from binary import deceptive
from binary import boundedly
from binary import royalroad8
from binary import mixed
from binary import bitstring
from binary import bit_mutation
from binary import bitstring_population
from sgoal import simple_crossover
from sgoal import transposition
from sgoal import init
from sgoal import best_evaluation
from sgoal import success_evaluation
from sgoal import stats
from sgoal import MAXIMIZE
from binary import GSC1
from binary import RMHC
from gabo import GABO

# Paper reported values
def report(sgoal, fx, iter, sr):
  sr *= 100/len(fx)
  avg, std = stats(fx)
  avg_iter, std_iter = stats(iter)
  print("Results for", sgoal)
  print("success rate: ", sr, "%", sep='')
  print("best: ", avg, "+/-", std, sep='')
  print("iter: ", avg_iter, "+/-", std_iter, sep='')
  print('*********************************')
  return avg, std, avg_iter, std_iter

#Global variables
testbed = [maxones, deceptive, boundedly, royalroad8, mixed] # Testbed
name = ['MaxOnes','GD3','GBD4','RR1','Mixed']
LENGTH = [100, 30, 40, 64, 100] # Bitstring length
MAX_EVALS = [205, 500, 1000, 6400, 5000] # Maximum number of function evaluations
OPTIMUM = [LENGTH[0], 10*LENGTH[1], LENGTH[2], LENGTH[3], 47*LENGTH[4]//20] # Optimum value of the associated test function
EXP = 100 # Number of experiments

for FUNCTION in range(len(testbed)): # Testing the deceptive function. Change the number accordingly
  print('***************', name[FUNCTION], '***************')
  fx = [] # Function value reached by the sgoal
  iter = [] # Iter when the best value is reached by the sgoal
  sr = 0 # Success rate of the sgoal

  #GABO experiment
  for i in range(EXP):
    init()
    y, fy = GABO(testbed[FUNCTION], MAX_EVALS[FUNCTION], bitstring(LENGTH[FUNCTION]))
    fx.append(fy)
    sr += 1 if success_evaluation(OPTIMUM[FUNCTION]) != -1 else 0
    iter.append(best_evaluation()[1])
  report('GABO', fx, iter, sr)

  #GSC1 experiment
  fx = [] 
  iter = [] 
  sr = 0
  for i in range(EXP):
    init()
    y, fy = GSC1(testbed[FUNCTION], MAX_EVALS[FUNCTION], bitstring(LENGTH[FUNCTION]))
    fx.append(fy)
    sr += 1 if success_evaluation(OPTIMUM[FUNCTION]) != -1 else 0
    iter.append(best_evaluation()[1])
  report('GSC1', fx, iter, sr)

  #RMHC experiment
  fx = [] 
  iter = [] 
  sr = 0
  for i in range(EXP):
    init()
    y, fy = RMHC(testbed[FUNCTION], MAX_EVALS[FUNCTION], bitstring(LENGTH[FUNCTION]))
    fx.append(fy)
    sr += 1 if success_evaluation(OPTIMUM[FUNCTION]) != -1 else 0
    iter.append(best_evaluation()[1])
  report('RMHC', fx, iter, sr)

*************** MaxOnes ***************
Results for GABO
success rate: 100.0%
best: 100.0+/-0.0
iter: 197.92+/-3.248630480679511
*********************************
Results for GSC1
success rate: 0.0%
best: 92.64+/-2.1564786110694447
iter: 203.0+/-0.0
*********************************
Results for RMHC
success rate: 0.0%
best: 93.24+/-2.0155396299750605
iter: 191.28+/-11.185776682912994
*********************************
*************** GD3 ***************
Results for GABO
success rate: 97.0%
best: 299.92+/-0.4833218389437821
iter: 331.03+/-124.52336768655107
*********************************
Results for GSC1
success rate: 1.0%
best: 267.62+/-9.172545993343393
iter: 379.72+/-151.26745056356313
*********************************
Results for RMHC
success rate: 0.0%
best: 285.06+/-2.9993999399879976
iter: 88.53+/-41.01815573621027
*********************************
*************** GBD4 ***************
Results for GABO
success rate: 93.0%
best: 39.89+/-0.44485952839070475
iter: 557.38+/-230.0945